In [26]:
import numpy as np
import pandas as pd

In [27]:
movies = pd.read_csv('movies_metadata.csv',low_memory=False)
movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [28]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [29]:
movies = movies[['id','original_title','original_language','vote_count']]
movies.rename(columns={
  'id':'movieId',
  'original_title':'title',
  'original_language':'language'
},inplace=True)
movies.head(3)

,movieId,title,language,vote_count
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0


In [30]:
ratings = ratings[['userId','movieId','rating']]
ratings.head(3)

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0


In [31]:
movies.isna().sum()

movieId        0
title          0
language      11
vote_count     6
dtype: int64

In [32]:
movies.dropna(inplace=True)
movies.isna().sum()

movieId       0
title         0
language      0
vote_count    0
dtype: int64

In [33]:
ratings['userId'].value_counts()

userId
45811     18276
8659       9279
270123     7638
179792     7515
228291     7410
          ...  
30155         1
9641          1
164717        1
243426        1
234625        1
Name: count, Length: 270896, dtype: int64

In [34]:
ratings.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [35]:
qt_ratings = ratings['userId'].value_counts() > 999
y = qt_ratings[qt_ratings].index
y.shape

(2509,)

In [36]:
ratings.shape

(26024289, 3)

In [37]:
ratings = ratings[ratings['userId'].isin(y)]
ratings.shape

(3844582, 3)

In [38]:
movies = movies[movies['vote_count']>999]


In [39]:
movies_language = movies['language'].value_counts()
movies_language

language
en    1100
fr       5
ja       5
it       3
ko       2
pt       1
de       1
es       1
cn       1
sv       1
id       1
Name: count, dtype: int64

In [40]:
movies = movies[movies['language']=='en']

In [41]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieId     1100 non-null   object 
 1   title       1100 non-null   object 
 2   language    1100 non-null   object 
 3   vote_count  1100 non-null   float64
dtypes: float64(1), object(3)
memory usage: 43.0+ KB


In [43]:
movies['movieId'] = movies['movieId'].astype('int32')

In [47]:
ratings_movies = ratings.merge(movies,on='movieId',how='inner')

In [48]:
ratings_movies.shape

(189882, 6)

In [49]:
ratings_movies.isna().sum()

userId        0
movieId       0
rating        0
title         0
language      0
vote_count    0
dtype: int64

In [50]:
ratings_movies.drop_duplicates(['userId', 'movieId'],inplace=True)


In [51]:
ratings_movies.shape

(189882, 6)

In [52]:
ratings_movies.drop(axis=1,columns=['movieId'])

,userId,rating,title,language,vote_count
0,229,1.0,Finding Nemo,en,6292.0
1,741,3.0,Finding Nemo,en,6292.0
2,1932,0.5,Finding Nemo,en,6292.0
3,3437,2.0,Finding Nemo,en,6292.0
4,3694,1.5,Finding Nemo,en,6292.0
...,...,...,...,...,...
189877,243443,1.0,Wanted,en,2583.0
189878,218368,4.0,The Hunger Games: Catching Fire,en,6656.0
189879,219677,1.0,RED 2,en,1548.0
189880,221327,4.0,Guardians of the Galaxy,en,10014.0


In [54]:
pivot_ratings = ratings_movies.pivot_table(columns='userId',index='title',values='rating')
pivot_ratings

userId,229,231,741,836,1104,1136,1243,1380,1652,1846,...,269632,269750,269913,270071,270123,270213,270237,270564,270654,270887
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.5,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN
12 Angry Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
127 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,NaN,NaN
2 Fast 2 Furious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wild Wild West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X-Men Origins: Wolverine,4.0,NaN,4.0,3.5,NaN,4.0,NaN,NaN,3.0,3.0,...,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,3.0,NaN
X-Men: Days of Future Past,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
pivot_ratings.fillna(0,inplace=True)

In [58]:
from scipy.sparse import csr_matrix


movies_sparse = csr_matrix(pivot_ratings)
type(movies_sparse)

scipy.sparse._csr.csr_matrix

In [73]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(algorithm='brute')
model.fit(movies_sparse)

NearestNeighbors(algorithm='brute')

In [74]:
distt, sugg = model.kneighbors(pivot_ratings.filter(items=['127 Hours'],axis=0).values.reshape(1,-1))

for i in range(len(sugg)):
  print(pivot_ratings.index[sugg[i]])

Index(['127 Hours', 'American Hustle', 'The Expendables 2', 'Lord of War',
       'RED 2'],
      dtype='object', name='title')


In [75]:
distt, sugg = model.kneighbors(pivot_ratings.filter(items=['Toy Story'],axis=0).values.reshape(1,-1))

for i in range(len(sugg)):
  print(pivot_ratings.index[sugg[i]])

Index(['Toy Story', 'Meet the Fockers', 'Top Gun',
       'Harry Potter and the Chamber of Secrets',
       'Austin Powers: International Man of Mystery'],
      dtype='object', name='title')


In [76]:
distt, sugg = model.kneighbors(pivot_ratings.filter(items=['1408'],axis=0).values.reshape(1,-1))

for i in range(len(sugg)):
  print(pivot_ratings.index[sugg[i]])

Index(['1408', 'Pirates of the Caribbean: At World's End', 'Platoon', 'Snitch',
       'The Expendables 2'],
      dtype='object', name='title')


In [77]:
distt, sugg = model.kneighbors(pivot_ratings.filter(items=['2 Fast 2 Furious'],axis=0).values.reshape(1,-1))

for i in range(len(sugg)):
  print(pivot_ratings.index[sugg[i]])

Index(['2 Fast 2 Furious', 'Bambi', 'The Matrix Reloaded',
       'Brokeback Mountain', 'Lord of War'],
      dtype='object', name='title')
